In [1]:
import sys

sys.path.append("../../src/stong_core_cython")
sys.path.append("../../")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import restricted_hartree_fock
import basis_data
from scipy.constants import physical_constants

from atom import *
from restricted_hartree_fock import *
from basis_data import *

hartree_to_ev_factor = physical_constants["Hartree energy in eV"][0]
df = pd.read_csv("../make_base/sto_ng/sto_3g.csv")
df_zetas = pd.read_csv("./zetas.csv")

In [3]:
r = 1.4632
atoms = [Atom("He", np.array([0.0, 0.0, 0.0])), Atom("H", np.array([0.0, 0.0, r]))]
basis_data = {
    "He": create_basis_data(df, [1.68], 1, 0),
    "H": create_basis_data(df, [1.2], 1, 0),
}
molecule = Molecule(atoms=atoms, basis=basis_data, charge=1)
hartreefock = RestrictedHartreeFock(molecule, max_iter=10000, tol=1e-10)
hartreefock.scf()
hartreefock.calculate_total_energy()
results = hartreefock.get_results()
print(results)
results["energy"] + 2.0 / r

Start SCF for Molecule(atoms=[Atom(symbol=He, atomic_number=2, position=[0. 0. 0.]), Atom(symbol=H, atomic_number=1, position=[0.     0.     1.4632])])
{'energy': -4.202847448645835, 'P': array([[0.77355363, 0.17273392],
       [0.17273392, 0.03857135]]), 'val': array([-1.63141845, -0.18900741]), 'vec': array([[-0.87951898,  0.80398631],
       [-0.1963959 , -1.17531964]])}


-2.8359803081318935

In [4]:
results

{'energy': -4.202847448645835,
 'P': array([[0.77355363, 0.17273392],
        [0.17273392, 0.03857135]]),
 'val': array([-1.63141845, -0.18900741]),
 'vec': array([[-0.87951898,  0.80398631],
        [-0.1963959 , -1.17531964]])}

In [5]:
np.array(hartreefock.ERI)

array([[[[1.049701  , 0.4471315 ],
         [0.4471315 , 0.58535082]],

        [[0.4471315 , 0.2274196 ],
         [0.2274196 , 0.36458074]]],


       [[[0.4471315 , 0.2274196 ],
         [0.2274196 , 0.36458074]],

        [[0.58535082, 0.36458074],
         [0.36458074, 0.74978643]]]])

In [6]:
charge = molecule.charge
norb = molecule.norb
val = results["val"]
vec = results["vec"]
eri = np.array(hartreefock.ERI)
t = 0
moeri = np.zeros((norb, norb, norb, norb))
moeri = np.einsum("pqrs,pi,qj,rk,sl->ijkl", eri, vec, vec, vec, vec)
omega = moeri[0, 1, 0, 1]
delta_t = -omega / (2 * val[1] - 2 * val[0])
delta_t
t = t + delta_t
print(omega)
print(t)

0.1417639269985373
-0.04914130668259425


In [7]:
delta_t * omega

-0.006966464613164023

In [8]:
omega = moeri[0, 1, 0, 1] + (2 * val[1] - 2 * val[0]) * t
print(omega)
delta_t = -omega / (2 * val[1] - 2 * val[0])
t = t + delta_t
t
t * moeri[0, 1, 0, 1]

0.0


-0.006966464613164023

In [9]:
def omega(t):
    coef0 = moeri[0, 1, 0, 1]
    coef1 = (
        2 * (val[1] - val[0])
        + moeri[0, 0, 0, 0]
        + moeri[1, 1, 1, 1]
        + 2 * (moeri[0, 1, 1, 0] - moeri[0, 0, 1, 1])
        - 2 * moeri[0, 0, 1, 1]
    )
    coef2 = -moeri[0, 1, 0, 1]
    return coef0 + coef1 * t + coef2 * t**2


def domegadt(t):
    coef1 = (
        2 * (val[1] - val[0])
        + moeri[0, 0, 0, 0]
        + moeri[1, 1, 1, 1]
        + 2 * (moeri[0, 1, 1, 0] - moeri[0, 0, 1, 1])
        - 2 * moeri[0, 0, 1, 1]
    )
    coef2 = -moeri[0, 1, 0, 1]
    return coef1 + 2 * coef2 * t

In [10]:
t = t - omega(t) / domegadt(t)
print(t)
print(t * moeri[0, 1, 0, 1])
t = t - omega(t) / domegadt(t)
print(t)
print(t * moeri[0, 1, 0, 1])
print(omega(t))

-0.06306917523254785
-0.00894093395352487
-0.06305699155238287
-0.008939206747179388
-2.1043733270699472e-11
